# Redis 1

# Lab: Redis

In [1]:
import redis

import json

import pprint

from datetime import timedelta

import time

import numpy as np


## Connect to the redis database; redis the container name is used for the host name; TCP port is 6379; databases are given a integer number instead of a name

In [2]:
cities_db = redis.Redis(host='redis', port=6379, db=0)

## Flush (delete all) from the cities database in case there are key / value pairs left from prior runs

In [3]:
cities_db.flushdb()

True

## Example of key / value pairs where both the key and the value are simple strings; the logic for key / value pairs is basically the same as a Python dictionary:  if the key does not exist, create a new "slot" with the key / value pair, if the key already exists, overwrite the value

In [4]:
cities_db.set("Berkeley", "California")
cities_db.set("Portland", "Oregon")
cities_db.set("Seattle", "Washington")
cities_db.set("Dallas", "Texas")
cities_db.set("Nashville", "Tennessee")
cities_db.set("Chicago", "Illinois")
cities_db.set("Miami", "Florida")

True

In [5]:
cities_db.keys()

[b'Chicago',
 b'Dallas',
 b'Berkeley',
 b'Portland',
 b'Seattle',
 b'Miami',
 b'Nashville']

In [6]:
cities_db.get("Berkeley")

b'California'

In [9]:
for key in cities_db.keys():
    print("key:", key, "value:", cities_db.get(key))

key: b'Chicago' value: b'Illinois'
key: b'Dallas' value: b'Texas'
key: b'Berkeley' value: b'California'
key: b'Portland' value: b'Oregon'
key: b'Seattle' value: b'Washington'
key: b'Miami' value: b'Florida'
key: b'Nashville' value: b'Tennessee'


In [10]:
cities_db.set("Berkeley", "CA")
cities_db.set("Portland", "OR")
cities_db.set("Seattle", "WA")
cities_db.set("Dallas", "TX")
cities_db.set("Nashville", "TN")
cities_db.set("Chicago", "IL")
cities_db.set("Miami", "FL")

True

In [11]:
for key in cities_db.keys():
    print("key:", key, "value:", cities_db.get(key))

key: b'Chicago' value: b'IL'
key: b'Dallas' value: b'TX'
key: b'Berkeley' value: b'CA'
key: b'Portland' value: b'OR'
key: b'Seattle' value: b'WA'
key: b'Miami' value: b'FL'
key: b'Nashville' value: b'TN'


## If we want to set a lot of values at the same time, we can use a feature called pipelining, which is more efficient than setting them one at a time; not until we execute the pipeline do the changes get made;

In [12]:
cities_pipe = cities_db.pipeline()

cities_list = [["Berkeley", "California"], 
               ["Portland", "Oregon"],
               ["Seattle", "Washington"],
               ["Dallas", "Texas"],
               ["Nashville", "Tennessee"],
               ["Chicago", "Illinois"],
               ["Miami", "Florida"]
              ]

for (city, state) in cities_list:
    cities_pipe.set(city, state)


In [13]:
for key in cities_db.keys():
    print("key:", key, "value:", cities_db.get(key))

key: b'Chicago' value: b'IL'
key: b'Dallas' value: b'TX'
key: b'Berkeley' value: b'CA'
key: b'Portland' value: b'OR'
key: b'Seattle' value: b'WA'
key: b'Miami' value: b'FL'
key: b'Nashville' value: b'TN'


In [14]:
cities_pipe.execute()

[True, True, True, True, True, True, True]

In [15]:
for key in cities_db.keys():
    print("key:", key, "value:", cities_db.get(key))

key: b'Chicago' value: b'Illinois'
key: b'Dallas' value: b'Texas'
key: b'Berkeley' value: b'California'
key: b'Portland' value: b'Oregon'
key: b'Seattle' value: b'Washington'
key: b'Miami' value: b'Florida'
key: b'Nashville' value: b'Tennessee'


## We can put an expirey on a key / value pair;  once the key / value pair expires, it is deleted; there are methods to specify time in milliseconds, seconds, minutes, specific date, time, etc.; set New York for 10 seconds, watch it disappear

In [16]:
cities_db.setex("New York", timedelta(seconds=10), "NY")

True

In [17]:
cities_db.get("New York")

b'NY'

In [19]:
# wait 10 seconds and see it disappear
cities_db.get("New York")

## We are been using simple strings, which are not that useful;  JSON values are much more useful and typical;  create a new database 1 (flush it to clean out from prior runs); load the temp_stores_nested.json using the store_id as the key and the JSON as a string as the value

In [20]:
stores_db = redis.Redis(host='redis', port=6379, db=1)

stores_db.flushdb()

stores_pipe = stores_db.pipeline()

In [21]:
f = open("temp_stores_nested.json")
file_json = json.load(f)
f.close()

stores_json_list = file_json["stores"]

In [22]:
for store in stores_json_list:
    stores_pipe.set(store["store_id"], json.dumps(store))
    
stores_pipe.execute()

[True, True, True, True, True]

In [23]:
# print only the first 90 characters of the JSON object, since they are large

for key in stores_db.keys():
    print("key:", key, "value:", stores_db.get(key)[:90])

key: b'4' value: b'{"store_id": 4, "street": "299 SE 3rd Ave", "city": "Miami", "state": "FL", "zip": "33131"'
key: b'5' value: b'{"store_id": 5, "street": "1202 Broadway", "city": "Nashville", "state": "TN", "zip": "372'
key: b'2' value: b'{"store_id": 2, "street": "1001 Broadway", "city": "Seattle", "state": "WA", "zip": "98122'
key: b'1' value: b'{"store_id": 1, "street": "3000 Telegraph Ave", "city": "Berkeley", "state": "CA", "zip": '
key: b'3' value: b'{"store_id": 3, "street": "2510 McKinney Ave", "city": "Dallas", "state": "TX", "zip": "75'


## Where redis really shines is that it is a scale out, in memory solution;  memory is backed to disk at periodic intervals, which can be changed in the config file;  one great example would be a leader board for a multi-player online game; we will simulate a multi-player game with random scores 

In [25]:
games_db = redis.Redis(host='redis', port=6379, db=2)

games_db.flushdb()

game_id = 3874297973

## Run the Jupyter Notebook redis_2 to get the set values (actually you can make several copies of redis_2 and run several at the same time!)

## Infine loop - to stop: menu bar => Kernet => Interrupt

In [26]:
scores = {}

base_score = 0

while (True):
    
    base_score += 50
    
    scores["Alice"] = base_score + np.random.randint(16,20)
    scores["Bob"] = base_score + np.random.randint(16,20)
    scores["Mary"] = base_score + np.random.randint(11,15)
    scores["Jeff"] = base_score + np.random.randint(11,15)
    scores["Roberta"] = base_score + np.random.randint(6,10)
    scores["Trevor"] = base_score + np.random.randint(6,10)
    scores["Wanda"] = base_score + np.random.randint(1,5)
    scores["Walter"] = base_score + np.random.randint(1,5)
    
    games_db.set(game_id, json.dumps(scores))
    
    time.sleep(1.0)
    

KeyboardInterrupt: 

## You try it

## Create database 3 for sales; load the file temp_sales_nested.json into the sales database with the receipt as the key and the JSON as the value

## Create database 4 for customers; load the file temp_customers_nested.json into the customers database with the customer_id as the key and the JSON as the value

In [32]:
sales_db = redis.Redis(host='redis', port=6379, db=3)

sales_db.flushdb()
sales_pipe = sales_db.pipeline()

In [29]:
f = open("temp_sales_nested.json")
file_json = json.load(f)
f.close()

sales_json_list = file_json["sales"]

In [33]:
for sale in sales_json_list:
    sales_pipe.set(sale["receipt_number"], json.dumps(sale))
    
sales_pipe.execute()

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [34]:
for key in sales_db.keys():
    print("key:", key, "value:", sales_db.get(key)[:90])

key: b'003-000164376' value: b'{"receipt_number": "003-000164376", "sale_date": "2020-07-22", "sub_total": 108, "tax": 0,'
key: b'001-000248269' value: b'{"receipt_number": "001-000248269", "sale_date": "2020-08-22", "sub_total": 60, "tax": 0, '
key: b'004-000183530' value: b'{"receipt_number": "004-000183530", "sale_date": "2020-09-02", "sub_total": 144, "tax": 0,'
key: b'004-000143587' value: b'{"receipt_number": "004-000143587", "sale_date": "2020-07-14", "sub_total": 60, "tax": 0, '
key: b'002-000115446' value: b'{"receipt_number": "002-000115446", "sale_date": "2020-05-03", "sub_total": 12, "tax": 0, '
key: b'003-000209184' value: b'{"receipt_number": "003-000209184", "sale_date": "2020-09-13", "sub_total": 36, "tax": 0, '
key: b'002-000217889' value: b'{"receipt_number": "002-000217889", "sale_date": "2020-08-23", "sub_total": 24, "tax": 0, '
key: b'002-000109083' value: b'{"receipt_number": "002-000109083", "sale_date": "2020-04-27", "sub_total": 132, "tax": 0,'
key: b'001-00016

In [ ]:
sales_db.get('001-000169216')